1. 挂载磁盘

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

2. 安装依赖的 pip 包

In [ ]:
! pip install pydub "git+https://github.com/openai/whisper.git" 

In [ ]:
from pydub import AudioSegment
import whisper
import torch
import pandas as pd

# 输入文件路径
input_file = "/content/drive/MyDrive/audio/bytetalk.mp3"

# 切分的时间间隔，单位为毫秒
segment_length = 30000

# 读取音频文件
audio = AudioSegment.from_file(input_file, format="mp3")

# 初始化whisper
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("medium", device=DEVICE)

# 计算总时间
total_length = len(audio)

# 初始化时间戳和识别结果列表
timestamps = []
results = []

# 循环切分音频
for i in range(0, total_length, segment_length):
    # 计算开始和结束时间
    start_time = i
    end_time = min(i + segment_length, total_length)

    # 截取音频片段
    segment = audio[start_time:end_time]

    # 生成文件名
    output_file = "/content/drive/MyDrive/result_data/{}.wav".format((i // segment_length + 1))

    # 导出音频片段到wav文件
    segment.export(output_file, format="wav")

    # 进行语音识别
    result = model.transcribe(output_file)
    print(result["text"])

    # 添加时间戳和识别结果到列表
    timestamps.append(start_time / 1000)
    results.append(result["text"])

# 合并时间戳和识别结果为DataFrame
df = pd.DataFrame({"timestamp": timestamps, "result": results})

# 输出到CSV文件
df.to_csv("/content/drive/MyDrive/result_data/output.csv", index=False)
